In [ ]:
import warnings
warnings.filterwarnings('ignore',category=RuntimeWarning)
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import glob,os,sys
from tqdm.auto import tqdm
import proplot as plot
import json,pickle
import dask.array as da
import gc
from sklearn.decomposition import PCA
sys.path.insert(1, '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/')
from tools import derive_var,read_and_proc,preproc_noensemble
from tools.mlr import mlr
from tools.preprocess import do_eof,preproc_maria,preproc_haiyan
%matplotlib inline
plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',
                'tick.labelweight':'ultralight','gridminor.linestyle':'--','title.weight':'normal','linewidth':0.5})

#### Read 

In [ ]:
path = '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/'
suffix = '_smooth_preproc_dict1b_g'
a = [read_and_proc.depickle(path+'TCGphy/testML/output/haiyan/processed/uvwheat/'+'mem'+str(lime)+suffix)['u'].shape for lime in tqdm(range(1,21))]
# divide experiments reference
divider = np.asarray([aobj[0] for aobj in a]).cumsum()

In [3]:
folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/testML/output/haiyan/processed/intermediate/'
pcastore = read_and_proc.depickle(folderpath+'pca/PCA'+'_'+'dict2_g')
dudvdwdth = read_and_proc.depickle(folderpath+'pca/dudvdwdth'+'_'+'dict2_g')
haiyan_data = read_and_proc.depickle(folderpath+'pca/flatvar'+'_'+'dict2_g')

#### Check components

In [1]:
def forward_diff(arrayin=None,delta=None,axis=None,LT=1):
    result = []
    if axis==0:
        for i in range(0,arrayin.shape[axis]-LT):
            temp = (arrayin[i+LT,:]-arrayin[i,:])/(LT*delta)
            result.append(temp)
        return np.asarray(result)
    
class datacheck:
    def __init__(self,pcadict=None,flatvardict=False,divider=None):
        #self.folderpath=folderpath
        self.pcadict = pcadict
        self.ctlflatvar = flatvardict
        self.divider = divider
        
    def _back_to_exp(self,timeseries=None,divider=None):
        printout = [timeseries[0:divider[0],:]]
        for i in range(1,19):
            printout.append(timeseries[divider[i-1]:divider[i],:])
        printout.append(timeseries[divider[-2]:,:])
        return printout
    
    def dudvdwVAR(self,dudvdw=None,vartest='w'):
        #dudvdw = read_and_proc.depickle(self.folderpath+dudvdwpath)
        timeseries = self.pcadict[vartest].transform(self.ctlflatvar[vartest])
        left_dot = [forward_diff(obj,60*60,0,1) for obj in self._back_to_exp(timeseries,self.divider)]
        left_dott = np.concatenate([obj for obj in left_dot],axis=0)
        for i in np.linspace(0,90,46):
            tempobj = np.dot(left_dott[:,0:int(i)],(self.pcadict[vartest].components_[0:int(i)]))
            print((np.var(tempobj)/np.var(dudvdw['d'+str(vartest)])).data)
        #TESTu_var = [np.var(obj)/np.var(dudvdw['d'+str(vartest)]) for obj in TESTu]
        del left_dot,left_dott
        gc.collect()
        return None

In [16]:
def myPCA_projection_sen(pca_dict=None,varname=None,toproj_flatvar=None,orig_flatvar=None):
    pca_orig = pca_dict[varname].transform(orig_flatvar)
    if pca_dict[varname].mean_ is not None:
        orig_mean = pca_dict[varname].mean_
    projvar_transformed = np.dot(toproj_flatvar-np.nanmean(toproj_flatvar,axis=0),pca_dict[varname].components_.T)
    del orig_mean
    gc.collect()
    return pca_orig, projvar_transformed

class input_output:
    def __init__(self,PCAdict=None,folderpath=None,ts_varname=None,nummem=None):
        self.PCAdict = PCAdict
        self.varname=ts_varname
        self.nummem = nummem # u: 36 (40% variability in du), v:16/32 (40% dv var;50%), w:44 (40% dw var)
    
    ###################################################################################################################################################
    # Produce time series
    ###################################################################################################################################################    
    def produce_timeseries(self,flatvar=None):
        ts_dict = {}
        for indx,obj in tqdm(enumerate(self.varname)):
            ts_dict[obj] = self.PCAdict[obj].transform(flatvar[obj].data)[:,0:self.nummem[indx]]
        return ts_dict
    
    def produce_Qsentimeseries(self,senvar_name=None,refvar_name='rad',numQ=None,flatvar=None,senflatvar=None):
        ts_dict = {}
        temp = [myPCA_projection_sen(pca_dict=self.PCAdict,varname=refvar_name,toproj_flatvar=senflatvar[obj].data,orig_flatvar=flatvar[refvar_name].data)[1][:,0:numQ] for indx,obj in enumerate(senvar_name)]
        return dict(zip(senvar_name,temp))
    
    def normalize_timeseries(self,timeseries=None):
        assert timeseries['u'].shape[-1]==26,"var shape error"
        ts_dict = {}
        for indx,obj in tqdm(enumerate(self.varname)):
            ts_dict[obj] = (timeseries[obj]-np.nanmean(timeseries[obj],axis=0))/np.nanstd(timeseries[obj],axis=0)
        return ts_dict
    
    def normalize_timeseries_decomp(self,sentimeseries=None,reftimeseries=None,senvarnames=None,refvarname='rad'):
        ts_dict = {}
        meanstd_dict = {}
        for indx,obj in tqdm(enumerate(senvarnames)):
            tempf = -np.mean(sentimeseries[obj],axis=0)/np.std(reftimeseries[refvarname],axis=0)
            ts_dict[obj] = (sentimeseries[obj]-np.mean(sentimeseries[obj],axis=0))/np.std(reftimeseries[refvarname],axis=0)
            meanstd_dict[obj] = np.broadcast_to(tempf, (sentimeseries[obj].shape[0], sentimeseries[obj].shape[1]))
        return ts_dict,meanstd_dict
    
    ###################################################################################################################################################
    # Produce Input Dataset
    ###################################################################################################################################################      
    def _back_to_exp(self,timeseries=None,divider=None):
        printout = [timeseries[0:divider[0],:]]
        for i in range(1,19):
            printout.append(timeseries[divider[i-1]:divider[i],:])
        printout.append(timeseries[divider[-2]:,:])
        return printout
    
    def back_to_exp(self,inputlong=None,divider=None,senvarname=None):
        ts_dict = {}
        if senvarname is None:
            for indx,obj in tqdm(enumerate(self.varname)):
                ts_dict[obj] = self._back_to_exp(inputlong[obj],divider)
        else:
            for indx,obj in tqdm(enumerate(senvarname)):
                ts_dict[obj] = self._back_to_exp(inputlong[obj],divider)            
        return ts_dict
    
    def train_valid_test(self,expvarlist=None,validindex=None,testindex=None,concat='Yes'):
        X_valid, X_test = [expvarlist[i] for i in validindex], [expvarlist[i] for i in testindex]
        X_traint = expvarlist.copy()
        popindex = validindex+testindex
        #[X_train.pop(i) for i in validindex]
        #[X_train.pop(i) for i in testindex]
        X_train = [X_traint[i] for i in range(len(X_traint)) if i not in popindex]
        assert len(X_train)==16, 'wrong train-valid-test separation!'
        if concat=='Yes':
            return np.concatenate([X_train[i] for i in range(len(X_train))],axis=0), np.concatenate([X_valid[i] for i in range(len(X_valid))],axis=0), np.concatenate([X_test[i] for i in range(len(X_test))],axis=0)
        else:
            return X_train, X_valid, X_test
    
    def make_X(self,expvarlist=None,varwant=None,validindex=None,testindex=None,concat='Yes'):
        trainlist,validlist,testlist = [],[],[]
        for obj in varwant:
            test1,test2,test3 = self.train_valid_test(exp_pca_norml[obj],validindex,testindex,'Yes')
            trainlist.append(test1)
            validlist.append(test2)
            testlist.append(test3)
        return np.concatenate([trainlist[i] for i in range(len(trainlist))],axis=1), np.concatenate([validlist[i] for i in range(len(validlist))],axis=1), np.concatenate([testlist[i] for i in range(len(testlist))],axis=1)
    
    ###################################################################################################################################################
    # Produce Output Dataset
    ###################################################################################################################################################
    def get_time_diff_terms(self,inputvar=None,LT=None):
        def _get_time_diff(array=None,timedelta=60*60,LT=None):
            store = []
            for exp in array:
                a = forward_diff(exp,timedelta,0,LT)
                if a.shape[0]>0:
                    azero = np.zeros((LT,exp.shape[-1]))
                    store.append(np.concatenate((a,azero),axis=0))
                else:
                    store.append(np.zeros((exp.shape[0],exp.shape[-1])))
            return store
        
        storedict = {}
        for wantvar in ['u','v','w','theta']:
            storedict[wantvar] = _get_time_diff(array=inputvar[wantvar],LT=LT)
        return storedict
    
    def make_Y(self,inputdict=None,LDT=None,validindex=[1,6],testindex=[2,12]):
        def _make_Y(inputt=None):
            varTRAIN,varVALID,varTEST = [],[],[]
            for varobj in ['u','v','w','theta']:
                test1,test2,test3 = self.train_valid_test(expvarlist=inputt[varobj],validindex=validindex,testindex=testindex,concat='Yes')
                varTRAIN.append(test1)
                varVALID.append(test2)
                varTEST.append(test3)
            return np.concatenate([varTRAIN[i] for i in range(len(varTRAIN))],axis=1), np.concatenate([varVALID[i] for i in range(len(varVALID))],axis=1), np.concatenate([varTEST[i] for i in range(len(varTEST))],axis=1)
        test = [self.get_time_diff_terms(inputdict,int(LDTobj)) for LDTobj in LDT]
        return [_make_Y(timediffobj) for timediffobj in test]

In [11]:
from tools.mlr import mlr,maria_IO,haiyan_IO
#datacheck(pcastore,haiyan_data,divider).dudvdwVAR(dudvdwdth,'w')
[np.abs(pcastore[obj].explained_variance_ratio_.cumsum()-0.91).argmin() for obj in ['u','v','w','qv','theta','heatsum','hdia','rad','ir']]

[8, 0, 10, 9, 3, 6, 6, 4, 3]

In [12]:
import random
rand_nums = random.sample(range(1,20),4)

In [18]:
nummem=[26,18,48,9,14,6,6,4,3]
haiyan_pca_nonorml = input_output( pcastore, None, ['u', 'v', 'w', 'qv','theta', 'heatsum', 'hdia', 'rad', 'ir'], nummem ).produce_timeseries( haiyan_data )
haiyan_pca_norml = input_output( pcastore, None, ['u', 'v', 'w', 'qv','theta', 'heatsum', 'hdia', 'rad', 'ir'], nummem ).normalize_timeseries( haiyan_pca_nonorml )

exp_pca_norml = input_output( pcastore, None, ['u', 'v', 'w', 'qv','theta', 'heatsum', 'hdia', 'rad', 'ir'], nummem ).back_to_exp( haiyan_pca_norml, divider ) 
exp_pca_nonorml = input_output( pcastore, None, ['u', 'v', 'w', 'qv','theta', 'heatsum', 'hdia', 'rad', 'ir'], nummem ).back_to_exp( haiyan_pca_nonorml, divider ) 

EXP = [['hdia','rad'],['theta','hdia','rad'],['u','v','w','theta','hdia','rad'],['u','v','theta','hdia','rad'],['u','v'],['u','v','w'],['u','v','hdia','rad'],['u','v','w','hdia','rad'],['u','v','w','theta','qv','hdia','rad']]
storename = ['dt','dtth','dtthuvw','dtthuv','uv','uvw','dtuv','dtuvw','dtthuvwqv']
import random

for i in tqdm(range(33)):
    rand_nums = random.sample(range(1,20),4)
    X_traindict,X_validdict,X_testdict = {},{},{}
    for ind,obj in enumerate(storename):
        test1,test2,test3 = (input_output(pcastore,None,['u','v','w','qv','theta','heatsum','hdia','rad','ir'],None).make_X(exp_pca_norml,EXP[ind],rand_nums[0:2],rand_nums[2:4],'Yes'))
        X_traindict[obj] = test1
        X_validdict[obj] = test2
        X_testdict[obj] = test3
    folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/testML/output/haiyan/processed/intermediate/'
    read_and_proc.save_to_pickle(folderpath+'pca/X/random/Xtrain'+str(rand_nums[0])+'_'+str(rand_nums[1])+'_'+str(rand_nums[2])+'_'+str(rand_nums[3])+'_'+'dict_g',X_traindict,'PICKLE')
    read_and_proc.save_to_pickle(folderpath+'pca/X/random/Xvalid'+str(rand_nums[0])+'_'+str(rand_nums[1])+'_'+str(rand_nums[2])+'_'+str(rand_nums[3])+'_'+'dict_g',X_validdict,'PICKLE')
    read_and_proc.save_to_pickle(folderpath+'pca/X/random/Xtest'+str(rand_nums[0])+'_'+str(rand_nums[1])+'_'+str(rand_nums[2])+'_'+str(rand_nums[3])+'_'+'dict_g',X_testdict,'PICKLE')
    
    y_all = (input_output(pcastore,None,['u','v','w','qv','theta','heatsum','hdia','rad','ir'],nummem).make_Y(exp_pca_nonorml,np.linspace(0,35,36)+1,rand_nums[0:2],rand_nums[2:4]))
    folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/testML/output/haiyan/processed/intermediate/'
    read_and_proc.save_to_pickle(folderpath+'pca/y/random/allY'+str(rand_nums[0])+'_'+str(rand_nums[1])+'_'+str(rand_nums[2])+'_'+str(rand_nums[3])+'_'+'dict_g',y_all,'PICKLE')

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]